## 다중회귀

농어의 길이만을 가지고 무게를 예측 했을 때 과소적합의 문제가 남아 있었다.
이를 해결하기 위해 더 고차항을 넣어야 하는데 길이만을 가지고 고차항을 추가하기가 쉽지 않다.

선형회귀는 특성이 많을수록 효과가 좋아지기 때문에 높이와 두께 등 다른 여러 데이터들이 있다면 효과를 높일 수 있다.


이번에는 3개의 특성(길이, 높이, 두께)을 사용할 것이다. 다항회귀에서 처럼 3개의 특성을 각각 제곱하고, 각 특성을 서로 곱해서 또 다른 특성을 만들어서 학습시켜보자.

이렇게 기존 특성을 사용해 새로운 특성을 뽑아내는 작업을 `특성공학(Feature Engineering)`이라고 한다.

#### Dataset

[농어의 특성(길이, 높이, 두께)](https://bit.ly/perch_csv_data)

[농어의 무게](http://bit.ly/perch_data)

In [7]:
import pandas as pd
df = pd.read_csv("perch.csv")
perch_full = df.to_numpy()

import numpy as np
perch_weight = np.array([5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 110.0,
       115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 130.0,
       150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 197.0,
       218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 514.0,
       556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 820.0,
       850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 1000.0,
       1000.0])

In [8]:
# 훈련세트와 테스트세트 분류

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(perch_full, perch_weight, random_state=42)

## 사이킷런의 변환기

- 사용할 변환기: `PolynomialFeatures()`
- 사용 예시 (2개의 특성(3, 4)으로 이루어진 샘플)
```python
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures()
poly.fit([[3, 4]])
print(poly.transform([[3, 4]]))
```
```
[[ 1.  3.  4.  9. 12. 16.]]
# 절편, 3, 4, 3^2, 3*4, 4^2
```

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(include_bias=False) # include_bias는 default=True, 사이킷런 모델은 자동으로 특성에 추가된 절편항을 무시하기 때문에 지정하지 않아도 문제 없음
poly.fit(train_input)
train_poly = poly.transform(train_input)

print(train_poly.shape)

(42, 9)


In [10]:
# 9개의 특성이 어떻게 만들어 졌는지 확인
poly.get_feature_names_out()

array(['x0', 'x1', 'x2', 'x0^2', 'x0 x1', 'x0 x2', 'x1^2', 'x1 x2',
       'x2^2'], dtype=object)

In [11]:
# 테스트 세트 변환
test_poly = poly.transform(test_input)

## 다중회귀 모델 훈련

In [13]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(train_poly, train_target)

LinearRegression()

In [ ]:
# 점수 확인
print(lr.score(train_poly, train_target))
print(lr.score(test_poly, test_target))

0.9903183436982124
0.9714559911594101


## 더 많은 특성 추가해보기
- 아래 코드를 확인하면 5제곱까지 특성을 만들어보면 특성의 개수가 56개나 되는 것을 확인할 수 있다.
- 절편항을 제외하더라도 55개나 있는 것이다.

In [21]:
# degree 매개변수를 사용하여 최대차수 지정
poly = PolynomialFeatures(degree=5)
poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)

print(train_poly.shape)

(42, 56)


In [22]:
# 모델 훈련
lr.fit(train_poly, train_target)

# 훈련세트 점수
lr.score(train_poly, train_target)

0.9999999999999999

In [23]:
# 테스트 세트 점수
lr.score(test_poly, test_target)

-144.40489791863044

- 특성의 개수를 크게 늘리면 선형모델은 아주 강력해지지만, 이런 모델은 훈련세트에 너무 과대적합 되어 테스트 세트에서는 형편없는 점수를 만들게 된다.
- 여기에 사용한 훈련세트 샘플의 개수는 42개 뿐이지만, 56개 특성으로 훈련하니 당연히도 훈련세트에 완벽하게 학습했다.
- 하지만, 훈련세트에 과도하게 학습되어 과대적합 문제를 해결하기 위해 특성을 줄여야 한다.